##Install Dependencies

In [1]:
! pip install -U langsmith openai ollama

In [2]:
import os
import openai
import pandas as pd
from google.colab import userdata

from langsmith import Client

In [9]:
#Manual Dataset creation
# 
inputs = [
    "How many tokens was DBRX pre-trained on?",
    "Is DBRX a MOE model and how many parameters does it have?",
    "How many GPUs was DBRX trained on and what was the connectivity between GPUs?",
]

outputs = [
    "DBRX was pre-trained on 12 trillion tokens of text and code data.",
    "Yes, DBRX is a fine-grained mixture-of-experts (MoE) architecture with 132B total parameters.",
    "DBRX was trained on 3072 NVIDIA H100s connected by 3.2Tbps Infiniband",
]

# Dataset
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

# Write to csv
csv_path = "/content/DBRX_eval.csv"
df.to_csv(csv_path, index=False)

In [19]:
# create langsmit client

client = Client()
dataset_name = "DBRX"
print(client)

Client (API URL: https://api.smith.langchain.com)


In [20]:
# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about DBRX model.",
)

In [21]:
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

In [25]:
datasets = client.list_datasets(dataset_name="DBRX")
print(list(datasets))

[Dataset(name='DBRX', description='QA pairs about DBRX model.', data_type=<DataType.kv: 'kv'>, id=UUID('fd714aca-5ff0-4c0e-8e27-0285603d2e19'), created_at=datetime.datetime(2024, 9, 2, 11, 6, 49, 859699, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2024, 9, 2, 11, 6, 49, 859699, tzinfo=datetime.timezone.utc), example_count=3, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None)]


In [26]:
# create a new project

os.environ["LANGCHAIN_PROJECT"] = "DBRX"

In [28]:
# data Load from blog post
import requests
from bs4 import BeautifulSoup

In [30]:
# load from URL
url = "https://www.databricks.com/blog/introducing-dbrx-new-state-art-open-llm"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
# print(soup)

In [31]:
text = [p.text for p in soup.find_all("p")]
full_text = "\n".join(text)
print(full_text)

by The Mosaic Research Team
March 27, 2024 in Mosaic AI Research
Today, we are excited to introduce DBRX, an open, general-purpose LLM created by Databricks. Across a range of standard benchmarks, DBRX sets a new state-of-the-art for established open LLMs. Moreover, it provides the open community and enterprises building their own LLMs with capabilities that were previously limited to closed model APIs; according to our measurements, it surpasses GPT-3.5, and it is competitive with Gemini 1.0 Pro. It is an especially capable code model, surpassing specialized models like CodeLLaMA-70B on programming, in addition to its strength as a general-purpose LLM.
This state-of-the-art quality comes with marked improvements in training and inference performance. DBRX advances the state-of-the-art in efficiency among open models thanks to its fine-grained mixture-of-experts (MoE) architecture. Inference is up to 2x faster than LLaMA2-70B, and DBRX is about 40% of the size of Grok-1 in terms of bot

In [36]:
# OpenAI API
import openai
from langsmith.wrappers import wrap_openai

openai_client = wrap_openai(openai.Client())

In [37]:
# create the question answers

def answer_dbrx_question_oai(inputs: dict) -> dict:
    """
    Generates answers to user questions based on a provided website text using OpenAI API.

    Parameters:
    inputs (dict): A dictionary with a single key 'question', representing the user's question as a string.

    Returns:
    dict: A dictionary with a single key 'output', containing the generated answer as a string.
    """

    # System prompt
    system_msg = (
        f"Answer user questions in 2-3 sentences about this context: \n\n\n {full_text}"
    )

    # Pass in website text
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": inputs["question"]},
    ]

    # Call OpenAI
    response = openai_client.chat.completions.create(
        messages=messages, model="gpt-3.5-turbo"
    )

    # Response in output dict
    return {"answer": response.dict()["choices"][0]["message"]["content"]}

In [38]:
# User question example

answer_dbrx_question_oai(
    {
        "question": "What are the main differences in training efficiency between MPT-7B vs DBRX?"
    }
)

{'answer': 'DBRX excels in training efficiency compared to MPT-7B. Specifically, a member of the DBRX family called DBRX MoE-A (7.7B total parameters) required 3.7x fewer FLOPs to reach a Databricks LLM Gauntlet score of 30.5% compared to MPT-7B (1T tokens). This efficiency improvement is attributed to better data quality, a more efficient architecture using MoE, optimization strategies, tokenization, and enhanced pretraining data.'}

In [39]:
# User question example

answer_dbrx_question_oai({"question": "How many tokens was DBRX pre-trained on?"})

{'answer': 'DBRX was pre-trained on 12 trillion tokens of text and code data.'}

In [47]:
# !pip install langchain

In [48]:
# create a Evaluator

from langsmith.evaluation import evaluate, LangChainStringEvaluator

# Evaluators
qa_evalulator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "DBRX"

ValueError: Evaluation with the <class 'langchain.evaluation.qa.eval_chain.CotQAEvalChain'> requires a language model to function. Failed to create the default 'gpt-4' model. Please manually provide an evaluation LLM or check your openai credentials.

In [ ]:
!pip install langchain

# create a Evaluator

from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain.evaluation.schema import StringEvaluator # import StringEvaluator from langchain

# Evaluators
qa_evalulator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "DBRX"

In [ ]:


experiment_results = evaluate(
    answer_dbrx_question_oai,
    data=dataset_name,
    evaluators=qa_evalulator,
    experiment_prefix="test-dbrx-qa-oai",
    # Any experiment metadata can be specified here
    metadata={
        "variant": "stuff website context into gpt-3.5-turbo",
    },
)